In [3]:
# Install necessary libraries
!sudo apt-get install python3-dev libmysqlclient-dev > /dev/null
!pip install mysqlclient > /dev/null
!sudo pip3 install -U sql_magic > /dev/null
!pip install psycopg2-binary  > /dev/null
!pip install -q geopandas sqlalchemy
!apt-get install python3-mysqldb
!pip install geopy
!pip install kmodes

Reading package lists... Done
Building dependency tree       
Reading state information... Done
python3-mysqldb is already the newest version (1.3.10-1build1).
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 20 not upgraded.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [4]:
!pip install streamlit -q

In [5]:
# Install the SQLAlchemy library if it is not installed
# !sudo apt-get install python3-dev libmysqlclient-dev > /dev/null
# !pip install mysqlclient > /dev/null
# !sudo pip3 install -U sql_magic > /dev/null
# !pip install psycopg2-binary > /dev/null

In [6]:
!ls

app.py	node_modules  nohup.out  package-lock.json  sample_data


In [7]:
%%writefile app.py
import streamlit as st
import pandas as pd
import numpy as np
from kmodes.kmodes import KModes
import matplotlib.pyplot as plt
from sqlalchemy import create_engine
import requests
import datetime
from bs4 import BeautifulSoup
from geopy.geocoders import Nominatim
import geopandas as gpd

conn_string = 'mysql://{user}:{password}@{host}:{port}/{db}?charset=utf8'.format(
  user='Non-Trivial', 
  password='N14GUb5l9B8=', 
  host = 'jsedocc7.scrc.nyu.edu', 
  port     = 3306, 
  encoding = 'utf-8',
  db = 'Non-Trivial'
)
engine = create_engine(conn_string)

def populate(df, column):
  genres = df[column].tolist()
  uniqueGenres = []
  for i in genres:
    for j in i.split(','):
      if j not in uniqueGenres:
        uniqueGenres.append(j.strip("' []"))

  foo = []
  for k in uniqueGenres:
    bar = []
    for l in range(0,len(df)):
      if k in df[column].iloc[l]:
        bar.append(1)
      else:
        bar.append(0)
    foo.append(bar)

  count = 0
  for i in uniqueGenres:
    df[i] = foo[count]
    count += 1

def cluster(df,num,column):
  kmode = KModes(n_clusters=num, init = "random", n_init = 5, verbose=1)
  clusters = kmode.fit_predict(df.iloc[:,column:])
  df.insert(0, "Cluster", clusters, True)

def findCategory(pivot, category):
  return pivot.idxmax(axis = 1)[category]

streaming_df = pd.read_sql_table("imdbStreaming", conn_string)
populate(streaming_df,"Genre")
cluster(streaming_df,15,5)

yelp_df = pd.read_sql_table("expandedYelp", conn_string)
populate(yelp_df,"food")
cluster(yelp_df,15,19)

clusteredStreaming = streaming_df.groupby(['Cluster']).sum().T.iloc[2:-1]
clusteredFood = yelp_df.groupby(['Cluster']).sum().T.iloc[6:]

header = st.container()
userInput = st.container()
visualization = st.container()


with header:
  st.title('What to Do in NYC')

with userInput:
  #sel_col, disp_col = st.columns(2)
  st.header('Enter Your Preferences')

  st.text('Movie Preferences')
  mpref1 = st.slider('Horror', min_value = 1.0, max_value = 10.0, step = 0.1)
  mpref2 = st.slider('Action', min_value = 1.0, max_value = 10.0, step = 0.1)
  mpref3 = st.slider('Comedy', min_value = 1.0, max_value = 10.0, step = 0.1)
  mpref4 = st.slider('Thriller', min_value = 1.0, max_value = 10.0, step = 0.1)
  mpref5 = st.slider('Romance', min_value = 1.0, max_value = 10.0, step = 0.1)
  mpref6 = st.slider('SciFi', min_value = 1.0, max_value = 10.0, step = 0.1)
  mpref7 = st.slider('Fantasy', min_value = 1.0, max_value = 10.0, step = 0.1)

  movie = {'Horror':mpref1, 'Action':mpref2, 'Comedy':mpref3, 'Thriller':mpref4, 'Romance':mpref5, 'Sci-Fi':mpref6, 'Fantasy':mpref7}

  st.text('Cuisine Preferences')
  fpref1 = st.slider('Italian', min_value = 1.0, max_value = 10.0, step = 0.1)
  fpref2 = st.slider('Mediterranean', min_value = 1.0, max_value = 10.0, step = 0.1)
  fpref3 = st.slider('Chinese', min_value = 1.0, max_value = 10.0, step = 0.1)
  fpref4 = st.slider('American', min_value = 1.0, max_value = 10.0, step = 0.1)
  fpref5 = st.slider('Korean', min_value = 1.0, max_value = 10.0, step = 0.1)
  fpref6 = st.slider('Latin', min_value = 1.0, max_value = 10.0, step = 0.1)

  food = {'italian':fpref1, 'mediterranean':fpref2, 'chinese':fpref3, 'newamerican':fpref4, 'korean':fpref5, 'latin':fpref6}

  User_Preferences = {'Movies':movie, 'Food':food, 'User Zipcode': 10003}

  @st.cache
  def topGenre():
    maxMovieScore = 0
    maxMovieGenre = ""
    for i in movie:
      if movie[i]>maxMovieScore:
        maxMovieScore = movie[i]
        maxMovieGenre = i
    return maxMovieGenre

  @st.cache
  def topCuisine():
    maxCuisineScore = 0
    maxCuisineType = ""
    for i in food:
      if food[i]>maxCuisineScore:
        maxCuisineScore = food[i]
        maxCuisineType = i
    return maxCuisineType

  geolocator = Nominatim(user_agent="ProjectsinProgramming")
  def findcoordinate(name):
    try:
      location = geolocator.geocode(name + " NYC")
      return location.latitude, location.longitude
    except:
      return None, None

  st.header("Click update to see recommendations based on your preferences!")
  if st.button('Update'):
    maxMovieGenre = topGenre()
    maxCuisineType = topCuisine()
    
    NumRecs = 3

    foodFilter = yelp_df["Cluster"] == findCategory(clusteredFood,maxCuisineType)
    userScoresFood = []
    for j in range(len(yelp_df)):
      userScore = 0
      for i in food:
        if (int(yelp_df.iloc[[j]][i].values[0])==1):
          userScore += food[i]
      userScore += yelp_df.iloc[[j]]["rating"].values[0]
      userScoresFood.append(userScore)
    yelp_df["User Score"] = userScoresFood
    sortedFoodCluster = yelp_df[foodFilter].sort_values(by=['User Score'], ascending=False)

    new = {'longitude': [], 'latitude': [],'Name': []}
    for i in range (0, NumRecs):
      try:
        st.text("Here's a restaurant that you would like: " + sortedFoodCluster.iloc[[i]]['name'].values[0])
        new['latitude'].append(sortedFoodCluster.iloc[i]['coordinates.latitude'])
        new['longitude'].append(sortedFoodCluster.iloc[i]['coordinates.longitude'])
        new['Name'].append(sortedFoodCluster.iloc[i]['name'])
      except:
        continue

      key = 'c68ma4janrpjmzkxxz4u9fs7'
      date = str(datetime.date.today()) #"2022-05-06"
      zip = str(sortedFoodCluster.iloc[[i]]['location.zip_code'].values[0])
      url = "http://data.tmsapi.com/v1.1/movies/showings?startDate=" + date + "&zip=" + zip + "&api_key=" + key
      resp = requests.get(url)
      data = resp.json()
      df = pd.json_normalize(data)
      theater_df = df[["tmsId", "rootId", "subType", "title", "releaseYear", "releaseDate", "genres", "longDescription", "shortDescription", "ratings", "runTime", "showtimes"]]
      list_codes = []
      for j in theater_df['ratings']:
        try:
          list_codes.append(j[0]['code'])
        except:
          list_codes.append("N/A")
      theater_df["rating"] = list_codes
      theater_df.drop('ratings', inplace=True, axis=1)
      theater_df['genres'] = theater_df['genres'].astype(str)
      resp = requests.get("https://www.imdb.com/showtimes/location/US/" + zip)
      soup = BeautifulSoup(resp.text)
      movies = soup.find_all("div", {'class':'lister-item mode-grid'})
      ratings = []
      for k,row in theater_df.iterrows():
        count = 0
        for movie in movies:
          if theater_df['title'].iloc[k] == movie.find('div', {'class': 'title'}).text:
            if movie.find('div',{'class':'inline-block ratings-imdb-rating'})== None:
              cool = 0
            else:  
              ratings.append(movie.find('div',{'class':'inline-block ratings-imdb-rating'}).text.strip())
              count += 1 
        if count == 0:
          ratings.append(None)
      theater_df.insert(12, 'ratings', ratings)
      theater_df['ratings'].astype(float)
      populate(theater_df,"genres")
      cluster(theater_df,10,13)
      theater_df.rename(columns = {'Science fiction':'Sci-Fi'}, inplace = True)
      clusteredTheater = theater_df.groupby(['Cluster']).sum().T.iloc[1:]

      theaterFilter = theater_df["Cluster"] == findCategory(clusteredTheater,maxMovieGenre)
      userScoresTheater = []
      for m in range(len(theater_df)):
          userScore = 0
          for n in User_Preferences["Movies"]:
            if (theater_df.iloc[[m]][n].values[0]==1):
              userScore += User_Preferences["Movies"][n]
          try:
            userScore += theater_df.iloc[[m]]["ratings"].values[0]
          except:
            userScore += 0
          userScoresTheater.append(userScore)
      theater_df["User Score"] = userScoresTheater
      sortedTheaterCluster = theater_df[theaterFilter].sort_values(by=['User Score'], ascending=False)
      
      for l in range (0, NumRecs):
        theaterList = sortedTheaterCluster.iloc[[i]]["showtimes"].tolist()
        try:
          st.text("You can catch " + sortedTheaterCluster.iloc[[l]]['title'].values[0] + " at " + theaterList[0][0]["theatre"]["name"] + " near " + sortedFoodCluster.iloc[[i]]['name'].values[0])
          latTheater, longTheater = findcoordinate(theaterList[0][0]["theatre"]["name"])
          new['latitude'].append(latTheater)
          new['longitude'].append(longTheater)
          new['Name'].append(theaterList[0][0]["theatre"]["name"])
        except:
          continue
      st.text("")

    new_df = pd.DataFrame.from_dict(new)
    shapefile = 'https://data.cityofnewyork.us/api/geospatial/cpf4-rkhq?method=export&format=GeoJSON'
    # Load the shapefile
    df_nyc = gpd.GeoDataFrame.from_file(shapefile)
    # Limit the data to only Manhattan neighborhoods 
    df_manhattan = df_nyc.query( "boro_name =='Manhattan' ")
    # Create a plot
    #manhattan_plot = df_manhattan.plot(linewidth=0.5, color='White', edgecolor='Black', figsize=(15, 10))

    base = df_manhattan.plot(linewidth=0.5, color='White', edgecolor='Black', figsize=(15, 10))

    clean_mask = (new_df.latitude > 40) & (new_df.latitude < 41) & (new_df.longitude < -73) & (
        new_df.longitude > -74.02)
    cleandf = new_df[clean_mask]

    z = cleandf.plot(kind='scatter', x='longitude', y='latitude', figsize=(15,10), ax = base, s=15,)

    for i,txt in enumerate(cleandf.Name):
      z.annotate(txt,(cleandf.longitude.iat[i]+0.00005,cleandf.latitude.iat[i]))
    
    st.pyplot(fig=plt)

    cleanName = []
    cleanLat = []
    cleanLong = []
    for i in range(len(new['Name'])):
      if new['Name'][i] not in cleanName:
        cleanName.append(new['Name'][i])
        cleanLat.append(new['latitude'][i])
        cleanLong.append(new['longitude'][i])
    for i in range(len(cleanName)):
      if cleanLat[i] == None :
        st.text("Unfortunately we do not have a location for " + cleanName[i])
    st.text("")

    streamingFilter = streaming_df["Cluster"] == findCategory(clusteredStreaming,maxMovieGenre)
    userScoresStreaming = []
    for j in range(len(streaming_df)):
      userScore = 0
      for i in User_Preferences["Movies"]:
        if (int(streaming_df.iloc[[j]][i].values[0])==1):
          userScore += User_Preferences["Movies"][i]
      try:
        userScore += streaming_df.iloc[[j]]["Rating"].values[0]
      except:
        userScore += 0
      userScoresStreaming.append(userScore)
    streaming_df["User Score"] = userScoresStreaming
    sortedStreamingCLuster = streaming_df[streamingFilter].sort_values(by=['User Score'], ascending=False)


    st.text("If you would like to binge a show or movie at home here are some streaming recommendations.")

    for i in range (0, NumRecs):
      try:
        st.text("You can watch " + sortedStreamingCLuster.iloc[[i]]['Title'].values[0] + " on " + sortedStreamingCLuster.iloc[[i]]['Platform'].values[0])
      except:
        continue

Overwriting app.py


In [8]:
!ls

app.py	node_modules  nohup.out  package-lock.json  sample_data


In [9]:
# Authenticate the NGROK
!pip install -q pyngrok
!ngrok authtoken 2JYqOFQMlVQMAO2f0LdhpJg108o_27RYbnQLjdBcUcK8PH2RF

Authtoken saved to configuration file: /root/.ngrok2/ngrok.yml


In [10]:
from pyngrok import ngrok

In [11]:
# To clear tunnels check and clear tunnels if necessary

# tunnels = ngrok.get_tunnels()
# tunnels
# list(map(lambda k: ngrok.disconnect(k.public_url),tunnels))

In [12]:
# Setup a tunnel to the streamlit port 8501
public_url = ngrok.connect(8501)
public_url

# After running the app, click on the first link below to access the app

<NgrokTunnel: "http://8540-34-86-107-176.ngrok.io" -> "http://localhost:8501">

In [13]:
# Run the application after ngrok tunnel is created
#!streamlit run app.py &>/dev/null &
!streamlit run /content/app.py




  You can now view your Streamlit app in your browser.

  Network URL: http://172.28.0.12:8501
  External URL: http://34.86.107.176:8501

Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 1, iteration: 1/100, moves: 15, cost: 95.0
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 2, iteration: 1/100, moves: 13, cost: 76.0
Run 2, iteration: 2/100, moves: 4, cost: 76.0
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 3, iteration: 1/100, moves: 9, cost: 79.0
Run 3, iteration: 2/100, moves: 0, cost: 79.0
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 4, iteration: 1/100, moves: 11, cost: 96.0
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 5, iteration: 1/100, moves: 16, cost: 96.0
Run 5, iteration: 2/100, moves: 2, cost: 96.0
Best run was number 2
/content/app.py:43: PerformanceWarning: DataFrame is highly fragm

In [14]:
# If ngrok isn't working use this.
#!npm install localtunnel

#!streamlit run app.py & !npx localtunnel --port 8501